In [ ]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!git checkout d426b58b9e32a2ffc8c8a1196143270e22054a46 # We are using transformers=2.4.1
!pip install .
!pip install -r ./examples/requirements.txt
%cd ..


fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
M	examples/run_multiple_choice.py
M	examples/utils_multiple_choice.py
HEAD is now at d426b58b9 Patch: v2.4.1
Processing /content/transformers
  Preparing metadata (setup.py) ... done
  Using cached tokenizers-0.0.11.tar.gz (30 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached boto3-1.33.10-py3-none-any.whl (139 kB)
  Using cached sentencepiece-0.1.99-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
  Using cached sacremoses-0.1.1-py3-none-any.whl (897 kB)
  Using cached botocore-1.33.10-py3-none-any.whl (11.8 MB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.8.2-py3-none-any.whl (82 kB)
  Created wheel for transformers: filename=transformers-2.4.1-py3-none-any.whl size=475170 sha256=847b7f81727a00550928a43f38d8d3ffd8233e7f3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/MyDrive'
#https://drive.google.com/drive/folders/1147ZJ7ZZs0InrNrUslUMG2X4FqFdkDJK link to download data
%cp -r /content/drive/MyDrive/cs682/project/data /content

Mounted at /content/drive


In [ ]:
!rm transformers/examples/run_multiple_choice.py
!rm transformers/examples/utils_multiple_choice.py
!cp /content/drive/MyDrive/cs682/project/run_multiple_choice.py transformers/examples/
!cp /content/drive/MyDrive/cs682/project/utils_multiple_choice.py transformers/examples/

In [ ]:
!mkdir output
!mkdir cache

mkdir: cannot create directory ‘output’: File exists
mkdir: cannot create directory ‘cache’: File exists


In [ ]:
import os

task = ['goal', 'step', 'order'][2]

model_type = ['bert', 'roberta', 'xlnet', 'gpt'][0]

In [ ]:
# from transformers import ALBERT_PRETRAINED_CONFIG_ARCHIVE_MAP
# from transformers import BERT_PRETRAINED_CONFIG_ARCHIVE_MAP
# !pip install transformers==2.0.0
os.environ['MODEL_TYPE'] = model_type
os.environ['MODEL_NAME'] = f'zharry29/{task}_benchmark_{model_type}'
os.environ['OUTPUT_DIR'] = f'./output/{task}_benchmark_{model_type}'
os.environ['DATA_DIR'] = f'./data/{task}/'
os.environ['CACHE_DIR'] = f'./cache'
!rm -rf $CACHE_DIR/*
!rm -rf zharry29/*


In [ ]:
# !git clone https://github.com/NVIDIA/apex
# %cd apex
# !python3 setup.py install
# %cd ..

# os.environ['MODEL_TYPE'] = model_type
# os.environ['MODEL_NAME'] = {'bert':'bert-base-uncased',
#                             'roberta':'roberta-base',
#                             'xlnet':'xlnet-base-cased',
#                             'gpt':'gpt2',
#                             }[model_type]
# os.environ['DATA_DIR'] = f'./data/{task}/'
# os.environ['OUTPUT_DIR'] = f'./output/{task}_benchmark_{model_type}'
# os.environ['CACHE_DIR'] = f'./cache'

In [ ]:
!python ./transformers/examples/run_multiple_choice.py \
  --task_name swag \
  --model_type $MODEL_TYPE \
  --model_name_or_path $MODEL_NAME \
  --do_eval \
  --data_dir $DATA_DIR \
  --max_seq_length 200 \
  --per_gpu_eval_batch_size=16 \
  --output_dir $OUTPUT_DIR \
  --overwrite_output \
  --cache_dir $CACHE_DIR \
  --fp16

2023-12-08 03:26:40.188755: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 03:26:40.188829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 03:26:40.188868: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 03:26:41.363008: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
12/08/2023 03:26:42 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
config.json: 100% 1.02k/1.02k [00:00<00:00, 4.93MB/s]
You passed along `num_labels=4` with an incom

## Sample implementation and results:
We consider a sample text goal and obtain all the images relevant to the goal by extracting the intent of the image with respect to the goal.
Now that we have all the relevant images and the textual description of each image, we move to the next task which is arranging all the relevant images in a coherent chronological order to achieve the goal. We simplify this by breaking it into smaller subtasks. We generate all possible pairs of
images and order each pair individually using existing pretrained models like BERT.

Now we test our step ordering code on a sample set consisting of 5 images. Using these 5 images we can generate 10 pairs of sequences.
All these 10 pairs are correctly ordered i.e, in each pair the first image is logically followed by the second image.
Now our model returns 0 if it predicts that the first image is followed by second one and vice versa. So for this given sample, if the model predicts 0 then it is working correctly.
After running the above code we get the
preds= [0 0 0 1 0 0 0 0 1 0] as compared to the
true output labels [0 0 0 0 0 0 0 0 0 0]

which implies that the model works with 80% accuracy in this case.

In [ ]:
# !git clone https://github.com/NVIDIA/apex
# %cd apex
# !python3 setup.py install
# %cd ..

Cloning into 'apex'...
remote: Enumerating objects: 11521, done.
remote: Counting objects: 100% (3658/3658), done.
remote: Compressing objects: 100% (506/506), done.
remote: Total 11521 (delta 3335), reused 3251 (delta 3149), pack-reused 7863
Receiving objects: 100% (11521/11521), 15.42 MiB | 22.86 MiB/s, done.
Resolving deltas: 100% (8094/8094), done.
/content/apex


torch.__version__  = 2.1.0+cu118


running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initia

In [ ]:
# os.environ['MODEL_TYPE'] = model_type
# os.environ['MODEL_NAME'] = {'bert':'bert-base-uncased',
#                             'roberta':'roberta-base',
#                             'xlnet':'xlnet-base-cased',
#                             'gpt':'gpt2',
#                             }[model_type]
# os.environ['DATA_DIR'] = f'./data/{task}/'
# os.environ['OUTPUT_DIR'] = f'./output/{task}_benchmark_{model_type}'
# os.environ['CACHE_DIR'] = f'./cache'
# !rm -rf $CACHE_DIR/* # If the disk quota is exceeded, Huggingface will fail to recognize a model name because of the failure of downloading the pretrained checkpoint
# !rm -rf zharry29/* # If the disk quota is exceeded, Huggingface will fail to recognize a model name because of the failure of downloading the pretrained checkpoint

# !python ./transformers/examples/run_multiple_choice.py \
#   --task_name swag \
#   --model_type $MODEL_TYPE \
#   --model_name_or_path $MODEL_NAME \
#   --do_train \
#   --do_eval \
#   --data_dir $DATA_DIR \
#   --learning_rate 5e-5 \
#   --num_train_epochs 3 \
#   --max_seq_length 200 \
#   --per_gpu_eval_batch_size=8 \
#   --per_gpu_train_batch_size=8 \
#   --gradient_accumulation_steps 2 \
#   --output_dir $OUTPUT_DIR \
#   --overwrite_output \
#   --cache_dir $CACHE_DIR \
#   --fp16

Streaming output truncated to the last 5000 lines.
Iteration:  35% 2635/7562 [15:21<28:29,  2.88it/s]
Iteration:  35% 2636/7562 [15:21<29:11,  2.81it/s]
Iteration:  35% 2637/7562 [15:22<28:25,  2.89it/s]
Iteration:  35% 2638/7562 [15:22<29:26,  2.79it/s]
Iteration:  35% 2639/7562 [15:22<28:36,  2.87it/s]
Iteration:  35% 2640/7562 [15:23<29:01,  2.83it/s]
Iteration:  35% 2641/7562 [15:23<28:13,  2.91it/s]
Iteration:  35% 2642/7562 [15:23<28:50,  2.84it/s]
Iteration:  35% 2643/7562 [15:24<28:06,  2.92it/s]
Iteration:  35% 2644/7562 [15:24<28:48,  2.85it/s]
Iteration:  35% 2645/7562 [15:24<28:05,  2.92it/s]
Iteration:  35% 2646/7562 [15:25<28:29,  2.88it/s]
Iteration:  35% 2647/7562 [15:25<27:51,  2.94it/s]
Iteration:  35% 2648/7562 [15:25<28:26,  2.88it/s]
Iteration:  35% 2649/7562 [15:26<27:49,  2.94it/s]
Iteration:  35% 2650/7562 [15:26<28:22,  2.89it/s]
Iteration:  35% 2651/7562 [15:26<27:53,  2.94it/s]
Iteration:  35% 2652/7562 [15:27<28:23,  2.88it/s]
Iteration:  35% 2653/7562 [15:2